<a href="https://colab.research.google.com/github/bhargav-joshi/NLP-Practicals/blob/main/Exp_5_NLP_POS_tagging_using_Hidden_Markov_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

sentences = input("")
#sentences = ['Mary Jane can see Will', 'Spot will see Mary','Will Jane spot Mary?', 'Mary will pat Spot' ]

taggedSentences = [[('mary', 'N'), ('jane', 'N'), ('can', 'M'), ('see', 'V'), ('will', 'N')],
                   [('spot', 'N'), ('will', 'M'), ('see', 'V'), ('mary', 'N')],[('Will', 'M'), ('jane', 'N'), ('spot', 'V'),
                  ('mary', 'N')],[('mary', 'N'), ('will', 'M'), ('pat', 'V'), ('spot', 'N')]]
wordOccurence = {}
tagOccurence = {}
transmissionProbability = []
emissionProbability = []

def print_table(tableHeading, headers, itemList, probabilityList):
  print('\n' + tableHeading + '\n')
  for header in headers:
    print('\t', header, end = ' ')
  print()
  for i in range(len(itemList)):
     print(itemList[i], end = ' ')
     probabilities = probabilityList[i]
     for probability in probabilities:
       print('\t', probability, end = ' ')
     print()
def get_occurences():
  for sentence in taggedSentences:
    for (word, tag) in sentence:
      if word.lower() not in wordOccurence:
        wordOccurence.update({word.lower(): 1})
      else:
        wordOccurence[word.lower()] += 1
      if tag not in tagOccurence:
        tagOccurence.update({tag: 1})
      else:
        tagOccurence[tag] += 1

def emission_probability():
  tagList = list(tagOccurence.keys())
  wordList = list(wordOccurence.keys())
  for word in wordList:
    wordTagCount = {'N': 0, 'M': 0, 'V': 0}
    for sentence in taggedSentences:
      for taggedWord in sentence:
        if taggedWord[0] == word:
          wordTagCount[taggedWord[1]] += 1
    emissions = [round(value / tagOccurence[tag], 2) for value, tag in zip(wordTagCount.values(), tagList)]
    emissionProbability.append(emissions)
  print_table('Emission Probabilities:', tagList, wordList, emissionProbability)
            
def form_tag_pairs():
  tagPairs = []
  for sentence in taggedSentences:
    for item in range(len(sentence) - 1):
      tagPairs.append(str(sentence[item][1] + ' ' + sentence[item+1][1]))
  return tagPairs

def count_tag_pairs(tagPairs):
  tagPairCount = {}
  for pair in tagPairs:
    if pair not in tagPairCount:
      tagPairCount.update({pair: 1})
    else:
      tagPairCount[pair] += 1
  return tagPairCount

def transmission_probability():
  tagList = list(tagOccurence.keys())
  tagPairs = form_tag_pairs()
  tagPairCount = count_tag_pairs(tagPairs)
  for tagA in tagList:
    transmission = []
    for tagB in tagList:
      try:
        value = tagPairCount[str(tagA + ' ' + tagB)] / tagOccurence[tagA]
      except KeyError as e:
        value = 0
      transmission.append(round(value, 2))
    transmissionProbability.append(transmission)
  print_table('Transmission Probabilities:', tagList, tagList, transmissionProbability)
  
def parse_corpus(corpus):
  print ('\n\nCorpus:', corpus)
  corpusTokens = word_tokenize(corpus)
  tagList = list(tagOccurence.keys())
  emittedTags = []
  print ('\n' + 'POS Tagging:')
  print ('Word\t Tag')
  for word in range(len(corpusTokens)):
    wordIndex = list(wordOccurence.keys()).index(corpusTokens[word])
    eProbability = emissionProbability[wordIndex]
    maxValue = 0
    maxTag = 0
    if word != 0:
      prevTag = emittedTags[word-1]
    else:
      prevTag = -1
    for i in range (3):
      probability = eProbability[i]
      if prevTag != -1:
        tProbability = transmissionProbability[prevTag][i]
      else:
        tProbability = 1
      value = probability + tProbability
      if value > maxValue:
        maxValue = value
        maxTag = i
    emittedTags.append(maxTag)
    tag = tagList[maxTag]
    print('{}\t {}'.format(corpusTokens[word], tag))
get_occurences()
for sentence in sentences:
  print (sentence)
emission_probability()
transmission_probability()
corpus = 'Jane will spot Will'
parse_corpus(corpus.lower())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Mary Jane can see Will Spot will see Mary Will Jane spot Mary? Mary will pat Spot
M
a
r
y
 
J
a
n
e
 
c
a
n
 
s
e
e
 
W
i
l
l
 
S
p
o
t
 
w
i
l
l
 
s
e
e
 
M
a
r
y
 
W
i
l
l
 
J
a
n
e
 
s
p
o
t
 
M
a
r
y
?
 
M
a
r
y
 
w
i
l
l
 
p
a
t
 
S
p
o
t

Emission Probabilities:

	 N 	 M 	 V 
mary 	 0.44 	 0.0 	 0.0 
jane 	 0.22 	 0.0 	 0.0 
can 	 0.0 	 0.25 	 0.0 
see 	 0.0 	 0.0 	 0.5 
will 	 0.11 	 0.5 	 0.0 
spot 	 0.22 	 0.0 	 0.25 
pat 	 0.0 	 0.0 	 0.25 

Transmission Probabilities:

	 N 	 M 	 V 
N 	 0.11 	 0.33 	 0.11 
M 	 0.25 	 0 	 0.75 
V 	 1.0 	 0 	 0 


Corpus: jane will spot will

POS Tagging:
Word	 Tag
jane	 N
will	 M
spot	 V
will	 N
